In [6]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans


# Load the adjacency matrix
file_path = '/Users/gianmarcoferrara/Desktop/adjacency_matrices/adjacency_matrix_1.csv'  # Replace with your actual file path
adjacency_matrix = pd.read_csv(file_path, header=None).values


# Create a directed graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)

# Calculate raw out-degree and in-degree
out_degree = {node: G.out_degree(node) for node in G.nodes()}
in_degree = {node: G.in_degree(node) for node in G.nodes()}

# Step 4: Calculate the difference between out-degree and in-degree using dictionary comprehension
degree_differences = {node: out_degree[node] - in_degree[node] for node in out_degree}

print(out_degree)
print(in_degree)

print(degree_differences)

{0: 3104, 1: 63, 2: 339, 3: 15, 4: 76, 5: 225, 6: 19, 7: 3, 8: 267, 9: 2, 10: 150, 11: 296, 12: 84, 13: 3, 14: 176, 15: 51, 16: 2, 17: 246, 18: 133, 19: 3, 20: 99, 21: 1, 22: 253, 23: 227, 24: 207, 25: 384, 26: 145, 27: 418, 28: 362, 29: 3, 30: 6, 31: 43, 32: 1, 33: 3, 34: 1, 35: 149, 36: 408, 37: 292, 38: 2, 39: 250, 40: 130, 41: 235, 42: 313, 43: 9, 44: 14, 45: 365, 46: 2, 47: 164, 48: 154, 49: 280, 50: 104, 51: 95, 52: 158, 53: 451, 54: 64, 55: 2, 56: 34, 57: 238, 58: 272, 59: 239, 60: 6, 61: 180, 62: 7, 63: 195, 64: 30, 65: 65, 66: 1, 67: 71, 68: 4, 69: 364, 70: 104, 71: 330, 72: 240, 73: 7, 74: 66, 75: 415, 76: 338, 77: 11, 78: 365, 79: 314, 80: 189, 81: 324, 82: 334, 83: 212, 84: 22, 85: 4, 86: 333, 87: 212, 88: 26, 89: 65, 90: 111, 91: 268, 92: 1, 93: 125, 94: 8, 95: 1, 96: 166, 97: 273, 98: 48, 99: 409, 100: 5, 101: 1, 102: 78, 103: 79, 104: 223, 105: 398, 106: 91, 107: 3, 108: 32, 109: 18, 110: 1, 111: 71, 112: 195, 113: 176, 114: 22, 115: 1, 116: 30, 117: 77, 118: 300, 119: 5